# STATE TWITTER TROLL DETECTION USING TRANSFORMERS

# INTRO + BACKGROUND

With the 2020 US election around the corner, the issue of [electoral interference](https://www.nytimes.com/2020/09/10/us/politics/russian-hacking-microsoft-biden-trump.html) by state actors via social media and other online means is back in the spotlight. Twitter was a major platform by which Russia sought to interfere with the 2016 US election, and few have doubts that Moscow, Beijing and others will return to the platform with new disinformation campaigns.

This series of notebooks will show an end-to-end walkthrough of how you can build a state Twitter troll detector by fine tuning a transformer model with a custom dataset. This builds on my earlier project using "classic" machine learning models and a simple bag-of-words approach to detect [state troll] tweets(https://github.com/chuachinhon/twitter_state_trolls_cch).

That earlier version worked well under certain conditions. If the model was trained on Russian troll tweets, for instance, it would be pretty good at picking out new, unseen Russian troll tweets which the model had not been trained on. But if you used it to try to pick out state troll tweets of Iranian or Venezuelian origin, the accuracy dips very significantly. Fuller discussion in my Medium post [here](https://towardsdatascience.com/using-data-science-to-uncover-state-backed-trolls-on-twitter-dc04dc749d69). 

The BOW approach is clearly outdated. I would expect the transformer models to perform much better. But can it generalise well? Meaning, in this particular context, whether a model fine tuned on Russian and Chinese state troll tweets can also effectively detect troll tweets by Iranian or Saudi operators? This is what I hope to find out at the end of the project.

## MODEL

For this project, I'll be fine tuning the [distilbert-base-uncased](https://huggingface.co/distilbert-base-uncased). Feel free to switch out to another model of your choice.


## CUSTOM DATASET

If you want to skip straight to the fine tuning part, go to notebook2.0 in this repo. I've provided the custom training(file name: [train_raw.csv](https://github.com/chuachinhon/transformers_state_trolls_cch/blob/master/data/train_raw.csv)) and validation datasets(file name: [validate.csv](https://github.com/chuachinhon/transformers_state_trolls_cch/blob/master/data/validate.csv)), and you can start from there. 

Notebooks 1.0-1.2 outline how I collected, assembled and re-sampled the custom dataset. I reckon not everyone will be interested or have the patience for this.


## RAW DATASETS/GROUND TRUTH

* Troll tweets: The state troll tweets used in this project are those flagged by Twitter, so that establishes our "ground truth" in terms of what constitutes a state troll tweet. The original CSV files are too big to be uploaded to Github, so you'll have to download them directly from [Twitter](https://transparency.twitter.com/en/reports/information-operations.html) if you wish to run this notebook or build a bigger sample. I've used six raw files from Twitter to build a sample set of 50K troll tweets. The six sets of state troll tweets comprise those by:

- Russia, 3 sets released in May 2020, Jan 2019, and Oct 2018.
- China, 3 sets released in May 2020, and Aug 2019.

* Real tweets: I scraped about 317K tweets from 175 verified users and accounts that I personally checked for authenticity. I've provided a sample of the Tweepy notebook, so feel free to scrape your own tweets if you prefer (you need your own auth keys). I won't provide the raw scraped files, but the concated CSV file with all 317k + files is here.


## REPO STRUCTURE

### 1. DATA FOLDER

* 5 CSV files for notebooks in this series. Note that raw troll tweet files from Twitter are not included here.

### 2. NOTEBOOKS FOLDER

* Notebooks 1.0 - 1.2: Data collection, cleaning and preparation. Optional if you just want to experiment with the final dataset.

* Notebooks 2.0 - 2.1: Fine tuning distilbert with custom dataset and detailed testing with unseen validation dataset, as well as a fresh dataset with state troll tweets from Iran.

* Notebook 3.0 - 3.1: Create and test optimised logistic regression and XGB models against datasets used to assess fine tuned Distilbert model.


### 3. APP FOLDER

* app.py + folders for "static" and "template: simple app for use on a local machine to demonstrate how a state troll tweet detector can be used in deployment. Unfortunately free hosting accounts can't accomodate the disk size required for pytorch and the fine tuned model, so I've not deployed this online. 


### 4. TROLL_DETECT FOLDER

* Fine tuned Distilbert model from Colab notebook2.0. Too big for Github, download [here](https://www.dropbox.com/sh/90h7ymog2oi5yn7/AACTuxmMTcso6aMxSmSiD8AVa) from Dropbox instead.

### 5. PKL FOLDER

* Pickled logistic regression model from notebook3.0 

* Pickled XGB model from notebook3.1 

# PART 1A: TROLL TWEETS COLLECTION, CLEANING AND PREPARATION

In this notebook, we'll deal with the state troll tweets as identified by Twitter. I'm only using those from China and Russia, as they are the two biggest state actors in this area.

In [ ]:
import numpy as np
import pandas as pd
import re
import csv

In [3]:
pd.set_option('display.max_columns', 40)

In [ ]:
# these datasets are huge and NOT in the repo. 
# Download them here: https://transparency.twitter.com/en/reports/information-operations.html

# Mainland Chinese state troll tweets
raw1 = pd.read_csv("../data/china_052020_tweets.csv").dropna(subset=["tweet_text"])
raw2 = pd.read_csv("../data/china_082019_1_tweets_csv_hashed.csv").dropna(
    subset=["tweet_text"]
)
raw3 = pd.read_csv("../data/china_082019_2_tweets_csv_hashed.csv").dropna(
    subset=["tweet_text"]
)

# Russian state troll tweets
raw4 = pd.read_csv("../data/russia_052020_tweets.csv").dropna(subset=["tweet_text"])
raw5 = pd.read_csv("../data/russian_linked_tweets_csv_hashed.csv").dropna(
    subset=["tweet_text"]
)
raw6 = pd.read_csv("../data/ira_tweets_csv_hashed.csv").dropna(subset=["tweet_text"])

china_raw = pd.concat([raw1, raw2, raw3])

russia_raw = pd.concat([raw4, raw5, raw6])


## 1.1: FILTER OUT NON-ENGLISH TWEETS AND RETWEETS

In [5]:
# I'm focusing only on English tweets. Retweets also filtered out

crit1 = china_raw["tweet_language"] == "en"
crit2 = china_raw["is_retweet"] == False
crit3 = ~china_raw["tweet_text"].str.startswith("RT@")

china_raw = china_raw[crit1 & crit2 & crit3].copy()

crit4 = russia_raw["tweet_language"] == "en"
crit5 = russia_raw["is_retweet"] == False
crit6 = ~russia_raw["tweet_text"].str.startswith("RT@")

russia_raw = russia_raw[crit4 & crit5 & crit6]


## 1.2: CLEAN TWEET TEXT

I'm removing links, mentions, digits, but keeping hashtags. Depending on your use case, you might want to revise the cleaning regex rules below.

I'm also dropping tweets which have fewer than 3 words after cleaning.

In [6]:
def clean_text(text):
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"\n", " ", text)
    text = re.sub(r"\'t", " not", text) # Change 't to 'not'
    text = re.sub(r'(@.*?)[\s]', ' ', text) # Remove @name
    text = re.sub(r"$\d+\W+|\b\d+\b|\W+\d+$", " ", text) # remove digits
    text = re.sub(r"[^\w\s\#]", "", text) #remove special characters except hashtags
    text = text.strip(" ")
    text = re.sub(' +',' ', text).strip() # get rid of multiple spaces and replace with a single
    return text

In [7]:
china_raw["clean_text"] = (
    china_raw["tweet_text"]
    .map(lambda text: clean_text(text))
)

russia_raw["clean_text"] = (
    russia_raw["tweet_text"]
    .map(lambda text: clean_text(text))
)


In [8]:
china_raw['word_count'] = china_raw['clean_text'].str.count(' ') + 1

russia_raw['word_count'] = russia_raw['clean_text'].str.count(' ') + 1

In [9]:
crit7 = ~china_raw["clean_text"].isnull()
crit8 = china_raw["clean_text"] != ""
crit9 = china_raw["word_count"] > 3

china_raw = china_raw[crit7 & crit8 & crit9].copy()

crit10 = ~russia_raw["clean_text"].isnull()
crit11 = russia_raw["clean_text"] != ""
crit12 = russia_raw["word_count"] > 3

russia_raw = russia_raw[crit10 & crit11 & crit12].copy()


In [10]:
cols = ['tweetid', 'user_display_name', 'tweet_text','clean_text']

china = china_raw[cols].copy()

russia = russia_raw[cols].copy()

In [11]:
# optional: if you want to save the full set for future samples
#china.to_csv("../data/china_trolls_full.csv", index=False)
#russia.to_csv("../data/russia_trolls_full.csv", index=False)

## 1.3: CREATE 50K TROLL TWEETS SAMPLE

To make the fine tuning process more manageable (time and resource-wise), I decided to sample a smaller slice of the fuller troll tweets data instead of using it in full. If you have access to a tonne of GPUs, feel free to use a bigger slice of the troll tweets data.

### For this project, troll tweets will be labelled 1, while real tweets will be labelled 0.

In [12]:
china_sample = china.sample(n=25000, random_state=42, replace=False)

russia_sample = russia.sample(n=25000, random_state=42, replace=False)

troll_sample = pd.concat([china_sample, russia_sample])


In [13]:
troll_sample["troll_or_not"] = 1

In [14]:
troll_sample.shape

(50000, 5)

In [15]:
troll_sample.head()

,tweetid,user_display_name,tweet_text,clean_text,troll_or_not
332962,1245883557362282497,85c9M6CDZxgBwoEye0rF12ZBgGl3xvz6Bnbvhp7MUKI=,"having each tiny wish come true, or having som...",having each tiny wish come true or having some...,1
417285,961577921461866496,曲剑明,＠null It is 12:25 UTC now,null It is UTC now,1
992674,941616158075211776,IFL1E0m0SRX2cdOtuLFV7xKtnBgxagKzNgkuGFvNtvs=,British number two Bedene to switch back to Sl...,British number two Bedene to switch back to Sl...,1
135543,850414479976345600,Klausv,kalamitykait Thanks for bearing with us - you ...,kalamitykait Thanks for bearing with us you sh...,1
463169,960784360071925760,曲剑明,＠null It is 08:56 CET now,null It is CET now,1


In [17]:
# I've included this dataset in the repo in case you want an even smaller slice of the dataset

"""
troll_sample.to_csv(
    "../data/troll_50k.csv",
    index=False,
    encoding="utf-8",
    quoting=csv.QUOTE_NONNUMERIC,
)
""" 